In [17]:
import os
import sklearn
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [7]:
FILE_ROOT = "C:/Users/tallu/Desktop/Project/"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "C:/Users/tallu/Desktop/Project/data/"
LABELS_ROOT = DATA_ROOT + "labels"
IMAGES_ROOT = DATA_ROOT + "images"  

DEST_IMAGES_PATH = "images"
DEST_LABELS_PATH = "labels" 

In [4]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [12]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

file name:  maksssksksss0.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss0.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss0.txt
file name:  maksssksksss1.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss1.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss1.txt
file name:  maksssksksss10.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss10.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss10.txt
file name:  maksssksksss100.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss100.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss100.txt
file name:  maksssksksss101.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss101.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss101.txt
file name:  maksssksksss102.xml
C:/Users/tallu/Desktop/Project/annotations/maksssksksss102.xml
C:/Users/tallu/Desktop/Project/data/labels/maksssksksss102.txt
file name:  maksssksksss103.xml
C:/Users/tallu/Desktop/Project/anno

In [11]:
def cord_converter(size, box):
    
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = cord_converter(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
from PIL import Image
import os, sys

path = ('C:/Users/tallu/Desktop/Project/images')

def resize():
    for item in os.listdir(path):
        if os.path.isfile(item):
            im = Image.open(item)
            f, e = os.path.splitext(item)
            imResize = im.resize((416,416), Image.ANTIALIAS)
            imResize.save(f + ' resized.jpg', 'JPEG', quality=90)

resize()

In [20]:
import cv2 
import os
import random
import glob

path = "C:/Users/tallu/Desktop/Project/data/images"

for index, filename in enumerate(glob.glob(path)):
    a = cv2.imread(filename)
    blur = cv2.GaussianBlur(a, (5, 5), random.randint(3, 10))
    basename = os.path.splitext(os.path.basename(filename))[0]
    cv2.imwrite(f'{basename}_Gaussian{index}.png')
    


In [19]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=7)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=8)

print('total count :',len(image_list))
print('train :',len(train_list))
print('validation   :',len(val_list))

total count : 853
train : 682
validation   : 85


In [ ]:
def move_data(file_list, img_labels_root, imgs_source, type):

    root_file = Path(DATA_ROOT + DEST_IMAGES_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)

    root_file = Path(DATA_ROOT + DEST_LABELS_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)

    for file in file_list:
        img_name = file.replace('.png', '')
        img_src_file = imgs_source + '/' + img_name + '.png'
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        # Copy image
        DICT_DIR = DATA_ROOT + DEST_IMAGES_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = DATA_ROOT + DEST_LABELS_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [ ]:
move_data(train_list, LABELS_ROOT, IMAGES_ROOT, "train")
move_data(val_list,   LABELS_ROOT, IMAGES_ROOT, "val")
